In [1]:
import pandas as pd
import json

In [2]:
from collections import defaultdict

In [3]:
from tqdm import tqdm_notebook as tqdm

In [4]:
import imp

### Load the instrument class ids

In [20]:
ont = json.load(open('../schema/ontology.json', 'r'))

In [21]:
omap = {o['id']: o for o in ont}

In [22]:
# ('Musical instrument', '/m/04szw')
instrument_classes = omap['/m/04szw']['child_ids']
instrument_classes.append('/m/015lz1') # Singing
instrument_classes.append('/m/01swy6') # Yodeling
instrument_classes.append('/t/dd00003') # Male singing
instrument_classes.append('/t/dd00004') # Female singing
instrument_classes.append('/t/dd00005') # Child singing

In [24]:
good_classes = []

In [25]:
candidates = [omap[i]['id'] for i in instrument_classes]

In [27]:
while candidates:
    new_class = candidates.pop()
    good_classes.append(new_class)
    candidates.extend(omap[new_class]['child_ids'])

In [28]:
good_classes = sorted(list(set(good_classes)))

In [29]:
class_map = {omap[i]['name']: i for i in good_classes}

In [30]:
instruments = [omap[i]['name'] for i in good_classes]

In [32]:
ns = imp.load_source('ns', '../scripts/namespaces.py')

In [33]:
openmic25_to_audioset_ids = defaultdict(list)

for k in ns.AUDIOSET_MAP:
    openmic25_to_audioset_ids[ns.AUDIOSET_MAP[k]].append(class_map[k])
    
openmic25_to_audioset_ids = dict(openmic25_to_audioset_ids)

In [34]:
ids_to_om25 = {}
for k in openmic25_to_audioset_ids:
    for aid in openmic25_to_audioset_ids[k]:
        ids_to_om25[aid] = k

In [35]:
good_ids = []
for k in openmic25_to_audioset_ids:
    good_ids.extend(openmic25_to_audioset_ids[k])

## Load a slice of the training data

In [36]:
df_bal = pd.read_csv('../data/audioset/balanced_train_segments.csv',
                     skiprows=3, names=['YTID', 'start', 'end', 'labels'], quoting=2, sep=',\s+')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [37]:
df_un = pd.read_csv('../data/audioset/unbalanced_train_segments.csv',
                     skiprows=3, names=['YTID', 'start', 'end', 'labels'], quoting=2, sep=',\s+')

/home/bmcfee/miniconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


In [38]:
inst_set = set(good_ids)

In [39]:
mapping = defaultdict(set)

for i, (ytid, start, end, labels) in tqdm(df_bal.iterrows(), desc='Balanced set', total=len(df_bal)):
    for tag in labels.strip('"').split(','):
        if tag in inst_set:
            mapping[ids_to_om25[tag]].add((ytid, start, end))

for i, (ytid, start, end, labels) in tqdm(df_un.iterrows(), desc='Unbalanced set', total=len(df_un)):
    for tag in labels.strip('"').split(','):
        if tag in inst_set:
            mapping[ids_to_om25[tag]].add((ytid, start, end))
            
mapping = dict(mapping)

HBox(children=(IntProgress(value=0, description='Balanced set', max=22160), HTML(value='')))

HBox(children=(IntProgress(value=0, description='Unbalanced set', max=2041789), HTML(value='')))

In [40]:
for k in mapping:
    mapping[k] = sorted(list(mapping[k]))

In [43]:
[(k, len(mapping[k])) for k in sorted(mapping)]

[('accordion', 2833),
 ('bagpipes', 1715),
 ('banjo', 2396),
 ('bass', 8549),
 ('cello', 5215),
 ('clarinet', 2061),
 ('cymbals', 5365),
 ('drums', 26331),
 ('flute', 4721),
 ('guitar', 56926),
 ('harmonica', 2156),
 ('harp', 1983),
 ('mallet_percussion', 7257),
 ('mandolin', 2312),
 ('organ', 3578),
 ('piano', 12744),
 ('saxophone', 3013),
 ('synthesizer', 4981),
 ('trombone', 2731),
 ('trumpet', 3771),
 ('ukulele', 5232),
 ('violin', 28065),
 ('voice', 59441)]

In [44]:
json.dump(mapping,
          open('/home/bmcfee/git/cosmir/dev-set-builder/data/audioset/openmic25_index.json', 'w'),
          indent=2)